In [1]:
import sys
from sys import platform
if platform  == 'win32':
    sys.argv=["_", r"P:\rmahajan14\capstone_data\data", r"P:\rmahajan14\capstone_data\cache"]
if platform == 'linux' or platform == 'linux2':
    def full_path(p):
        return os.path.abspath(os.path.expanduser(p))
    sys.argv=["_", r"/mnt/d/DiDi_Ride_Data/data", r"/mnt/d/DiDi_Ride_Data/Ride_Data_Cache"]

In [2]:
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

In [3]:
from loader1 import *
from utils import *
from common import *
from experiment2 import *
# from sys import platform

import warnings
warnings.filterwarnings('ignore')

In [4]:
import time

Basic Coding 

In [21]:
start = '2016-11-01'
end = '2016-11-30'
orders = merge_order_df(start=start, end=end, use_cache=True)

../Ride_Data_Cache/merged_orders.msgpack exists


In [18]:
orders.shape

(6104724, 15)

In [22]:
pool_rides(orders)
orders['shifted_pool'] = orders['is_pool'].shift(1)
orders['shifted_pool'] = orders['shifted_pool'] & orders['is_pool'] & (orders['driver_id'] == orders['shifted_driver'])

In [23]:
print(orders[(orders.shifted_pool == True)].shape)
orders[(orders.shifted_pool == True)].head(3)

(16600, 15)


,order_id,ride_start_timestamp,ride_stop_timestamp,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,ride_duration,driver_id,shifted_end_time,shifted_driver,cond_1,cond_2,is_pool,shifted_pool
33721,2c51748e1a0a5f045d54d6889c49fe93,2016-11-10 17:19:32,2016-11-10 17:33:10,104.083655,30.665329,104.079831,30.653490,13.633333,0001860739024029fa3da2cad0ed4de2,2016-11-10 18:52:51,0001860739024029fa3da2cad0ed4de2,True,True,True,True
156213,dc90a199ba3767ee64c7b24c0835811d,2016-11-01 20:53:13,2016-11-01 21:51:06,104.111010,30.669780,104.056580,30.531100,57.883333,000211a97ba2b768ef941593bf1ae1a5,2016-11-01 21:10:00,000211a97ba2b768ef941593bf1ae1a5,True,True,True,True
86143,6d8208557a463b3f40f4df34666dfbca,2016-11-22 14:37:57,2016-11-22 14:56:02,104.049070,30.713680,104.068718,30.762031,18.083333,000219b0a1e9f864f311767d683524d0,2016-11-22 15:05:46,000219b0a1e9f864f311767d683524d0,True,True,True,True


In [26]:
from scipy.sparse.csgraph import connected_components

def reductionFunction(data):
    # create a 2D graph of connectivity between date ranges
    start = data.ride_start_timestamp.values
    end = data.ride_stop_timestamp.values
    graph = (start <= end[:, None]) & (end >= start[:, None])

    # find connected components in this graph
    n_components, indices = connected_components(graph)

    # group the results by these connected components
    return data.groupby(indices).aggregate({'ride_start_timestamp': 'min',
                                            'ride_stop_timestamp': 'max'})

new_val = orders.groupby(['driver_id']).apply(reductionFunction).reset_index('driver_id')
    

In [17]:
orders[orders.driver_id == '000211a97ba2b768ef941593bf1ae1a5']

,order_id,ride_start_timestamp,ride_stop_timestamp,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,ride_duration,driver_id,shifted_end_time,shifted_driver,cond_1,cond_2,is_pool,shifted_pool
85996,7a4833b8e39db563365e485f3c16feb5,2016-11-01 09:29:41,2016-11-01 09:45:47,104.01616,30.67141,104.060430,30.670650,16.100000,000211a97ba2b768ef941593bf1ae1a5,2016-11-02 03:15:30,00020e3d875210ba7b86de9a17a969d9,True,False,False,False
35848,32eb295835b821f4b01c9eb9292bd534,2016-11-01 20:06:55,2016-11-01 20:28:25,104.05099,30.63277,104.118610,30.661450,21.500000,000211a97ba2b768ef941593bf1ae1a5,2016-11-01 09:45:47,000211a97ba2b768ef941593bf1ae1a5,False,True,False,False
19718,1bd834f316b12c5f76848a7382c554e9,2016-11-01 20:43:34,2016-11-01 21:24:56,104.12024,30.66279,104.136750,30.609250,41.366667,000211a97ba2b768ef941593bf1ae1a5,2016-11-01 20:28:25,000211a97ba2b768ef941593bf1ae1a5,False,True,False,False
8323,0bb417e0e260876d3c7b6d7afc7e0c7f,2016-11-01 20:49:50,2016-11-01 21:10:00,104.11256,30.66713,104.116592,30.642699,20.166667,000211a97ba2b768ef941593bf1ae1a5,2016-11-01 21:24:56,000211a97ba2b768ef941593bf1ae1a5,True,True,True,False
156213,dc90a199ba3767ee64c7b24c0835811d,2016-11-01 20:53:13,2016-11-01 21:51:06,104.11101,30.66978,104.056580,30.531100,57.883333,000211a97ba2b768ef941593bf1ae1a5,2016-11-01 21:10:00,000211a97ba2b768ef941593bf1ae1a5,True,True,True,True


In [12]:
orders.head(2)

,order_id,ride_start_timestamp,ride_stop_timestamp,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,ride_duration,driver_id,shifted_end_time,shifted_driver,cond_1,cond_2,is_pool
175165,f727597b61fc2f0141aadecf2a3b5ee0,2016-11-01 11:57:05,2016-11-01 12:13:18,104.12148,30.64421,104.07986,30.65953,16.216667,0000131d486b69eb77ab6e9e7cca9f4c,NaT,NaN,False,False,False
90347,804a1a47f2b10dc36b7e909d9b977f02,2016-11-01 12:33:05,2016-11-01 12:49:04,104.13995,30.64042,104.08466,30.66191,15.983333,0000131d486b69eb77ab6e9e7cca9f4c,2016-11-01 12:13:18,0000131d486b69eb77ab6e9e7cca9f4c,False,True,False


In [5]:
spatial_type = 'radial'

In [24]:
df_final_no_decay, target_df_no_decay = get_final_df_reg(use_cache=True, decay='No Decay', spatial_type=spatial_type)

file path is ../Ride_Data_Cache/order_20161101.msgpack
file path is ../Ride_Data_Cache/order_20161102.msgpack
file path is ../Ride_Data_Cache/order_20161103.msgpack
file path is ../Ride_Data_Cache/order_20161104.msgpack
file path is ../Ride_Data_Cache/order_20161105.msgpack
file path is ../Ride_Data_Cache/order_20161106.msgpack
file path is ../Ride_Data_Cache/order_20161107.msgpack
file path is ../Ride_Data_Cache/order_20161108.msgpack
file path is ../Ride_Data_Cache/order_20161109.msgpack
file path is ../Ride_Data_Cache/order_20161110.msgpack
file path is ../Ride_Data_Cache/order_20161111.msgpack
file path is ../Ride_Data_Cache/order_20161112.msgpack
file path is ../Ride_Data_Cache/order_20161113.msgpack
file path is ../Ride_Data_Cache/order_20161114.msgpack
file path is ../Ride_Data_Cache/order_20161115.msgpack
file path is ../Ride_Data_Cache/order_20161116.msgpack
file path is ../Ride_Data_Cache/order_20161117.msgpack
file path is ../Ride_Data_Cache/order_20161118.msgpack
file path 

In [25]:
df_final_old_decay, target_df_old_decay = get_final_df_reg(use_cache=True, decay='Old Decay',spatial_type=spatial_type)

../Ride_Data_Cache/merged_orders.msgpack exists
orders
Decay Calculation
Old Decay
Creating the exponential decay
Dumping to ../Ride_Data_Cache/idle_times_old.msgpack
Decay Calculation done in 53.90809917449951
1e
../Ride_Data_Cache/features_orders.msgpack exists
Features created in 0.8068151473999023
1f
../Ride_Data_Cache/radial_spatial_df.msgpack exists
spatial
Spatial Calculation done in 3.6069700717926025


In [26]:
# Using the function create_modified_active_time_through_decay2 
# instead of create_modified_active_time_through_decay
df_final_new_decay1, target_df_new_decay1 = get_final_df_reg(use_cache=True, decay='New Decay', mult_factor=1, spatial_type=spatial_type)
df_final_new_decay10, target_df_new_decay10 = get_final_df_reg(use_cache=True, decay='New Decay', mult_factor=10, spatial_type=spatial_type)
df_final_new_decay100, target_df_new_decay100 = get_final_df_reg(use_cache=True, decay='New Decay', mult_factor=100, spatial_type=spatial_type)

../Ride_Data_Cache/merged_orders.msgpack exists
orders
Decay Calculation
New Decay
Creating the exponential decay
Dumping to ../Ride_Data_Cache/idle_times_new_1.msgpack
Decay Calculation done in 54.064675092697144
1e
../Ride_Data_Cache/features_orders.msgpack exists
Features created in 0.8336372375488281
1f
../Ride_Data_Cache/radial_spatial_df.msgpack exists
spatial
Spatial Calculation done in 3.5006508827209473
../Ride_Data_Cache/merged_orders.msgpack exists
orders
Decay Calculation
New Decay
Creating the exponential decay
Dumping to ../Ride_Data_Cache/idle_times_new_10.msgpack
Decay Calculation done in 57.3330340385437
1e
../Ride_Data_Cache/features_orders.msgpack exists
Features created in 0.8815009593963623
1f
../Ride_Data_Cache/radial_spatial_df.msgpack exists
spatial
Spatial Calculation done in 4.443143844604492
../Ride_Data_Cache/merged_orders.msgpack exists
orders
Decay Calculation
New Decay
Creating the exponential decay
Dumping to ../Ride_Data_Cache/idle_times_new_100.msgpack

In [144]:
# Using the function create_modified_active_time_through_decay2 
# instead of create_modified_active_time_through_decay
df_final_new_decay_surv, target_df_new_decay_surv = get_final_df_reg(use_cache=True, decay='Survival', spatial_type=spatial_type)

../Ride_Data_Cache/merged_orders.msgpack exists
orders
Decay Calculation
Survival
Creating the Survival Functions
Dumping to ../Ride_Data_Cache/survival_probability_df.msgpack
Decay Calculation done in 53.466230154037476
1e
../Ride_Data_Cache/features_orders.msgpack exists
Features created in 0.811682939529419
1f
../Ride_Data_Cache/radial_spatial_df.msgpack exists
spatial
Spatial Calculation done in 4.3115150928497314


### Modelling Code 

We have till now coded the following models 

1. Linear Regression 
2. Linear Regression with L1 Regularization
3. Linear Regression with L2 Regularization
4. Random Forest Regressor 

Now we split the data in a ratio of 75:25

In [29]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV, ElasticNet
from sklearn.ensemble import RandomForestRegressor

In [30]:
def build_model(df, target_df, name, seed=0):
    # regression forest
    print(f'building regression model for {name}')
    X = df    
    xtrain, xtest, ytrain, ytest = train_test_split(X, target_df['target'], random_state=seed)

    sc = StandardScaler()

    xtrain_sc = sc.fit_transform(xtrain)

    rr = RandomForestRegressor(random_state=seed)
    rr.fit(xtrain_sc, ytrain)

    xtest_sc = sc.transform(xtest)
    
    print(rr.score(xtest_sc, ytest))
    
    return rr.score(xtest_sc, ytest)

In [164]:
df_final_new_decay_surv.shape, target_df_new_decay_surv.shape
# df_final_no_decay.shape, target_df_no_decay.shape

((1181102, 112), (1181102, 5))

In [34]:
df_list = [df_final_no_decay, df_final_old_decay, df_final_new_decay1, 
           df_final_new_decay10, df_final_new_decay100]
target_df_list = [target_df_no_decay, target_df_old_decay, target_df_new_decay1, 
                  target_df_new_decay10, target_df_new_decay100]
name_list = ['no_decay', 'old_decay', 'new_decay1', 'new_decay10', 'new_decay100']

In [166]:
df_list_updated = [df_final_new_decay_surv]
target_df_list_updated = [target_df_new_decay_surv]
name_list_updated = ['survival']

In [35]:
result = {}
for df, target_df, name in zip(df_list, target_df_list, name_list):
    result[name] = build_model(df, target_df, name)

building regression model for no_decay
0.487284067717064
building regression model for old_decay
0.6757007478822953
building regression model for new_decay1
0.6662170568641492
building regression model for new_decay10
0.749684039129092
building regression model for new_decay100
0.7914960448891515


In [167]:
for df, target_df, name in zip(df_list_updated, target_df_list_updated, name_list_updated):
    result[name] = build_model(df, target_df, name)

building regression model for survival
0.888966343003949


In [168]:
result

{'no_decay': 0.487284067717064,
 'old_decay': 0.6757007478822953,
 'new_decay1': 0.6662170568641492,
 'new_decay10': 0.749684039129092,
 'new_decay100': 0.7914960448891515,
 'survival': 0.888966343003949}

In [30]:
col = [x.shape[1] for x in df_list]
col

[111, 112, 112, 112, 112]

In [31]:
set(df_list[1].columns).difference(df_list[0].columns)

{'inactive_time'}

In [28]:
df_final_old_decay.head()

,num_total_rides,num_pool_rides,% of pool rides,"(0 days 00:00:44.999999, 0 days 10:36:30]_count","(0 days 10:36:30, 0 days 14:43:06]_count","(0 days 14:43:06, 0 days 18:45:25]_count","(0 days 18:45:25, 0 days 23:59:59]_count","(0 days 00:00:44.999999, 0 days 10:36:30]_sum","(0 days 10:36:30, 0 days 14:43:06]_sum","(0 days 14:43:06, 0 days 18:45:25]_sum",...,"(9509.75, 78107.176](-38.414, -8.521]","(9509.75, 78107.176](-8.521, 14.323]","(9509.75, 78107.176](-99.688, -38.414]","(9509.75, 78107.176](111.762, 137.56]","(9509.75, 78107.176](137.56, 180.0]","(9509.75, 78107.176](14.323, 38.147]","(9509.75, 78107.176](38.147, 60.214]","(9509.75, 78107.176](60.214, 88.383]","(9509.75, 78107.176](88.383, 111.762]",inactive_time
driver_id,,,,,,,,,,,,,,,,,,,,,
0000131d486b69eb77ab6e9e7cca9f4c,3,0.0,0.000000,0,1,0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
00002724a19c5f6a54ae8d60a378997e,1,0.0,0.000000,1,0,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
000042288ba964aa04d14869bd56efb6,5,0.0,0.000000,0,0,0,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.690378
000049d47b79747ad9927ba1bf5f4126,11,1.0,0.090909,0.363636,0.0909091,0.545455,0,0.347145,0.083383,0.569472,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,243.061737
00005adc52a78e1630cfb50e4bd0f89e,4,0.0,0.000000,0,0,0,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [29]:
df_final_new_decay10.head()

,num_total_rides,num_pool_rides,% of pool rides,"(0 days 00:00:44.999999, 0 days 10:36:30]_count","(0 days 10:36:30, 0 days 14:43:06]_count","(0 days 14:43:06, 0 days 18:45:25]_count","(0 days 18:45:25, 0 days 23:59:59]_count","(0 days 00:00:44.999999, 0 days 10:36:30]_sum","(0 days 10:36:30, 0 days 14:43:06]_sum","(0 days 14:43:06, 0 days 18:45:25]_sum",...,"(9509.75, 78107.176](-38.414, -8.521]","(9509.75, 78107.176](-8.521, 14.323]","(9509.75, 78107.176](-99.688, -38.414]","(9509.75, 78107.176](111.762, 137.56]","(9509.75, 78107.176](137.56, 180.0]","(9509.75, 78107.176](14.323, 38.147]","(9509.75, 78107.176](38.147, 60.214]","(9509.75, 78107.176](60.214, 88.383]","(9509.75, 78107.176](88.383, 111.762]",inactive_time
driver_id,,,,,,,,,,,,,,,,,,,,,
0000131d486b69eb77ab6e9e7cca9f4c,3,0.0,0.000000,0,1,0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.871019
00002724a19c5f6a54ae8d60a378997e,1,0.0,0.000000,1,0,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
000042288ba964aa04d14869bd56efb6,5,0.0,0.000000,0,0,0,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,75.475753
000049d47b79747ad9927ba1bf5f4126,11,1.0,0.090909,0.363636,0.0909091,0.545455,0,0.347145,0.083383,0.569472,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,333.463457
00005adc52a78e1630cfb50e4bd0f89e,4,0.0,0.000000,0,0,0,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,27.870189
